In [1]:
import numpy as np
from utils import *

import keras.utils
from keras import backend as K
from keras.datasets import mnist
from keras.models import save_model,load_model,Model
from keras.optimizers import Adam, RMSprop
from keras.layers import Conv2D, UpSampling2D, AveragePooling2D, MaxPooling2D, Dense,Input, Dropout
from keras.layers import LeakyReLU,Reshape,BatchNormalization, Flatten
from keras.models import save_model
from tempfile import TemporaryFile
import tensorflow as tf
import pickle
import scipy.io as sio
import matplotlib.pyplot as plt
from IPython import display
import time
from sklearn.mixture import GaussianMixture
import umap
from copy import deepcopy
from PIL import Image
from sklearn.cluster import KMeans
from scipy.io import loadmat
import scipy as sc
from skimage.color import rgb2gray
from skimage.transform import rescale, resize 

Using TensorFlow backend.


In [2]:
# ! pip install --user umap-learn
# ! pip uninstall --user keras
# ! pip install --user keras==2.1.6
# ! pip install --user matplotlib
# ! pip install --user np_utils
#! pip install --user  Pillow
#! pip install --user  scikit-image 

In [3]:
epochs = 6000
epochstep = epochs/100
nofclasses=10
batchsize=200 
nofprojections = 100

## Data Generation

In [4]:
#Read MNIST

(x_train,y_train),(x_test,y_test)=mnist.load_data()
dataY_train=np.expand_dims(x_train.astype('float32')/255.,3)
dataY_test=np.expand_dims(x_test.astype('float32')/255.,3)

labelY_train = keras.utils.to_categorical(y_train, nofclasses)
labelY_test = keras.utils.to_categorical(y_test, nofclasses)

In [5]:
numdataTrain = labelY_train.shape[0]
numdataTest = labelY_test.shape[0]

dataY_train1 = np.reshape(dataY_train,[numdataTrain,28*28]) 
dataY_test1 = np.reshape(dataY_test,[numdataTest,28*28]) 

In [6]:
USPSMat  = []
USPSTar  = []
curPath  = r'/nas/home/mrostami/NeurIPS2020/SVHNdataset/'
savedImg = []

train = loadmat(curPath+'train_32x32.mat')
test = loadmat(curPath+'test_32x32.mat')

# train and test are python dictionaries
# keys are ['__header__', '__version__', '__globals__', 'X', 'y']

X_train = train['X']
y_train = train['y']
X_test = test['X']
y_test = test['y']

X_train = np.rollaxis(X_train, 3)
X_test = np.rollaxis(X_test, 3)

In [7]:
y_train1 = y_train
y_test1 = y_test
y_train = np.zeros(y_train1.shape[0])
y_test = np.zeros(y_test.shape[0])

for i in range(y_test.shape[0]):
    y_test[i] = y_test1[i][0]
    if y_test[i]==10:
        y_test[i] = 0
    
for i in range(y_train.shape[0]):
    y_train[i] = y_train1[i][0]
    if y_train[i]==10:
        y_train[i] = 0

In [8]:
labelX_train = keras.utils.to_categorical(y_train, nofclasses)
labelX_test = keras.utils.to_categorical(y_test, nofclasses)


In [9]:
dataX_train = np.zeros([y_train1.shape[0],28,28,1])
dataX_test = np.zeros([y_test1.shape[0],28,28,1])


for i in range(X_train.shape[0]):
    dataX_train[i,:,:,0] =  resize(rgb2gray(X_train[i,:,:,0]),[28,28])
    
for i in range(X_test.shape[0]):
    dataX_test[i,:,:,0] =  resize(rgb2gray(X_test[i,:,:,0]),[28,28])

###  X is input and Z the embedding space

In [10]:
imgX = Input(shape=(28, 28, 1), name="input_img")  # adapt this if using `channels_first` image data format
labelX=K.placeholder(shape=(None,nofclasses),dtype='float32') #labels of input images oneHot

## Encoder and Classifier NN

In [11]:
def encoderNN(imgX,nofclasses):
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(imgX)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='block3_pool')(x)
    '''
    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='block4_pool')(x)
    # Block 5
    x = Conv2D(512, (2, 2), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (2, 2), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (2, 2), activation='relu', padding='valid', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    '''
    # Top layers
    x = Flatten(name='flatten')(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)




    embed = Dense(nofclasses, activation='relu')(x)
    encoderX = Model(imgX, embed)

    return encoderX

    



In [12]:
encoderX = encoderNN(imgX,nofclasses)

 





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
def    classifierNN(nofclasses):
    yin =  Input(shape=( nofclasses,) )

    probX = Dense(nofclasses, activation='softmax')(yin)


    classifier=Model(inputs=[yin],outputs=[probX])
    return classifier


In [14]:
classifier = classifierNN(nofclasses)

# Training on Task 1

In [15]:

discriminationLoss=K.mean(K.categorical_crossentropy(labelX,classifier(encoderX(imgX)))) 

params=encoderX.weights  + classifier.weights  

myLoss = discriminationLoss

opt = Adam(lr=1e-4) # very important
updates = opt.get_updates(myLoss,params)
train = K.function(inputs=[imgX,labelX],outputs=[discriminationLoss],updates=updates)





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [16]:
print(keras.__version__) # 2.1.6

2.1.6


In [17]:
print(tf.__version__) # 1.15.0

1.15.0


In [18]:
loss = []
 

In [19]:
for itr in range(epochs):
    indTrainDataX,trainLabelX=batchGenerator(labelX_train,batchsize,nofclasses=10)
    trainDataX=dataX_train[indTrainDataX,...]

    loss.append(train(inputs=[trainDataX, trainLabelX ]))
 

In [20]:
perd_label_X = classifier.predict(encoderX.predict(dataX_test))
print(100*float(sum(1*(np.argmax(perd_label_X,axis=1)==np.argmax(np.squeeze(labelX_test),axis=1))))/perd_label_X.shape[0])



92.67824216349109


In [21]:
gmmX = encoderX.predict(dataX_train)
gmmY = np.argmax(labelX_train,axis=1)


In [22]:
yper = classifier.predict(encoderX.predict(dataX_train))
yper = np.argmax(yper, axis=1)

In [23]:
gmmX = gmmX[gmmY==yper,:]
gmmY = gmmY[gmmY==yper]

In [24]:
gmmModel =  GaussianMixture(n_components=nofclasses,covariance_type='full', max_iter=300,init_params='kmeans', tol=1e-04) 


In [25]:
gmmModel.fit(gmmX,gmmY)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=300,
                means_init=None, n_components=10, n_init=1,
                precisions_init=None, random_state=None, reg_covar=1e-06,
                tol=0.0001, verbose=0, verbose_interval=10, warm_start=False,
                weights_init=None)

In [26]:
gmmModelSingle =  GaussianMixture(n_components=1,covariance_type='full') 

for i in range(nofclasses):
    a= gmmX[gmmY==i,:]
    gmmModelSingle.fit(a)
    gmmModel.weights_[i] = .1  
    gmmModel.covariances_[i] = gmmModelSingle.covariances_[0]
    gmmModel.means_[i] = gmmModelSingle.means_[0]
    
    gmmModel.precisions_cholesky_[i] = gmmModelSingle.precisions_cholesky_[0]
    gmmModel.precisions_[i] = gmmModelSingle.precisions_[0]
    
    
    
    
    

# Target Task

In [27]:
imgY = Input(shape=(nofclasses,) )   
labelY=K.placeholder(shape=(None,nofclasses),dtype='float32') #labels of input images oneHot
lamda2=1e-2

theta=K.variable(generateTheta(nofprojections,nofclasses),) 


 
discriminationLoss= K.mean(K.categorical_crossentropy(labelY,classifier(imgY))) 
 
matchingLoss=sWasserstein(encoderX(imgX),imgY,theta,nclass=nofclasses,Cp=None,Cq=None,)


myLoss= discriminationLoss +lamda2*matchingLoss 


params=encoderX.weights  + classifier.weights  
opt = Adam(lr=1e-4)  
updates = opt.get_updates(myLoss,params)
train = K.function(inputs=[imgX,imgY,labelY,theta],outputs=[lamda2*matchingLoss],updates=updates)

  


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/nas/home/mrostami/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [28]:
thres = .99
epochs2 = 20 * 100
epochstep = epochs2/100

for itr in range(epochs2):
    indTrainDataY,trainLabelY=batchGenerator(labelY_train,batchsize,nofclasses=10)
    trainDataY=dataY_train[indTrainDataY,...]
    
   
    Yembed1 = np.zeros([batchsize,nofclasses])
    Yembed = np.zeros([batchsize,nofclasses])
    Yembedlabel = np.zeros([batchsize,nofclasses])
    
    Yembed,Yembedlabel1  = gmmModel.sample(n_samples=batchsize)
    
    Yembedlabel = keras.utils.to_categorical(Yembedlabel1)   
    
    
    theta_=generateTheta(nofprojections,nofclasses)
    loss.append(train(inputs=[trainDataY,Yembed,Yembedlabel,theta_]))
     
    
        
        


In [29]:
perd_label_Y = classifier.predict(encoderX.predict(dataY_test))
print(100*float(sum(1*(np.argmax(perd_label_Y,axis=1)==np.argmax(np.squeeze(labelY_test),axis=1))))/perd_label_Y.shape[0])



93.56
